In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from urllib.parse import urlparse, parse_qs
from datetime import datetime
from dateutil import parser

In [2]:
from datetime import datetime

date_str = '01-10-2023'
date_object = datetime.strptime(date_str, '%d-%m-%Y')
print(date_object)
day = date_object.day
month = date_object.strftime('%B')  # %B gives the full month name
year = date_object.year

print(f"day={day}")
print(f"month={month}")
print(f"year={year}")


2023-10-01 00:00:00
day=1
month=October
year=2023


In [3]:
# url='https://www.krjogja.com/indeks-berita?daterange=15%20October,%202023%20-%2015%20October,%202023'
# headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
# }
# response = requests.get(url,headers=headers)
# soup = BeautifulSoup(response.text, "html.parser")
# cek = soup.find('div', {"class": "paging__wrap clearfix"})
# links = cek.find_all('a')
# all_link = []
# for link in links:
#     if link and 'href' in link.attrs:
#         link_href = link['href']
#         all_link.append(link_href)
# last_link = all_link[-2]
# # Split the URL using '&' as a delimiter

# url_parts = last_link.split('&')

# # Loop through the parts of the URL to find the one containing 'page='
# page_number = None
# for part in url_parts:
#     if 'page=' in part:
#         # Extract the page number by splitting at '='
#         page_number = int(part.split('=')[1])

# print(page_number)



In [4]:
def scrape_links(page_number, date):
    # Convert date to string explicitly
    cek_date = datetime.strptime(date, "%Y-%m-%d")
    day = cek_date.day
    month = cek_date.strftime('%B')  # %B gives the full month name
    year = cek_date.year
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.krjogja.com/indeks-berita?daterange={day}%20{month}%20{year}%20-%20{day}%20{month}%20{year}&page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h2', {"class": "latest__title"})

    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)

    print(f"Scraped {len(page_links)} links from page {page_number} url {url}")
    return page_links

In [5]:
def scrape_link_per_day(date, max_threads=5):
    cek_date = datetime.strptime(date, "%Y-%m-%d")
    day = cek_date.day
    month = cek_date.strftime('%B')  # %B gives the full month name
    year = cek_date.year
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.krjogja.com/indeks-berita?daterange={day}%20{month}%20{year}%20-%20{day}%20{month}%20{year}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    cek = soup.find('div', {"class": "paging__wrap clearfix"})
    links = cek.find_all('a')
    all_link = []
    for link in links:
        if link and 'href' in link.attrs:
            link_href = link['href']
            all_link.append(link_href)
    last_link = all_link[-2]

    # Extract the "page" parameter value from the last link
    url_parts = last_link.split('&')

    # Loop through the parts of the URL to find the one containing 'page='
    page_value = None
    for part in url_parts:
        if 'page=' in part:
            # Extract the page number by splitting at '='
            page_value = int(part.split('=')[1])
    # Check if page_value is an integer
    try:
        page_number = page_value
    except (TypeError, ValueError):
        page_number = 1

    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        # Use list comprehension to submit tasks to the thread pool
        futures = [executor.submit(scrape_links, index, date) for index in range(1, page_number + 1)]

        # Collect results from completed futures
        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [6]:
scrape_link_per_day('2023-11-01')

Scraped 10 links from page 1 url https://www.krjogja.com/indeks-berita?daterange=1%20November%202023%20-%201%20November%202023&page=1
Scraped 10 links from page 2 url https://www.krjogja.com/indeks-berita?daterange=1%20November%202023%20-%201%20November%202023&page=2
Scraped 10 links from page 5 url https://www.krjogja.com/indeks-berita?daterange=1%20November%202023%20-%201%20November%202023&page=5
Scraped 10 links from page 3 url https://www.krjogja.com/indeks-berita?daterange=1%20November%202023%20-%201%20November%202023&page=3
Scraped 10 links from page 4 url https://www.krjogja.com/indeks-berita?daterange=1%20November%202023%20-%201%20November%202023&page=4
Scraped 9 links from page 7 url https://www.krjogja.com/indeks-berita?daterange=1%20November%202023%20-%201%20November%202023&page=7
Scraped 10 links from page 6 url https://www.krjogja.com/indeks-berita?daterange=1%20November%202023%20-%201%20November%202023&page=6


['https://www.krjogja.com/yogyakarta/1243210259/dkki-diy-gelar-kejuaraan-perdana-demi-kembangkan-full-contact-karate',
 'https://www.krjogja.com/nasional/1243209258/laba-awan-hingga-september-2023-sebesar-rp-103-juta',
 'https://www.krjogja.com/nasional/1243209909/menteri-lhk-siti-nurbaya-evaluasi-efektifitas-kerja-urusan-konkurensi-lhk-di-daerah-oikn',
 'https://www.krjogja.com/solo/1243208713/polres-sukoharjo-rayonisasi-antisipasi-gangguan-kamtibmas-pemilu-2024',
 'https://www.krjogja.com/belanja/1243209873/momogi-ajak-anak-anak-berkreasi-lewat-kompetisi-tiktok-dance-challenge-dan-photo-contest',
 'https://www.krjogja.com/otomotif/1243209806/sambut-akhir-tahun-astra-motor-yogyakarta-bagi-hadiah-lewat-aplikasi-motorku-x',
 'https://www.krjogja.com/kulonprogo/1243209767/rutan-kelas-iib-wates-gelar-tes-narkoba-ini-hasilnya',
 'https://www.krjogja.com/peristiwa/1243209753/hps-2023-pse-kwi-launching-revitalisasi-wujudkan-pangan-berkeadilan-sosial-ekonomi-dan-ekologi',
 'https://www.krjogj

In [7]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time

def scrape_url(url, max_retries=2):
    retries = 0
    while retries < max_retries:
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
            }
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Judul Berita
                title_elem = soup.find('h1', {"class": "read__title"})
                title_text = title_elem.text.strip() if title_elem else "Title not found"

                # tanggal berita
                date_elem = soup.find('div', {"class": "read__info__date"})
                date_text = date_elem.text.strip() if date_elem else "Date not found"

                # Translation dictionary for Indonesian to English
                translation_dict = {
                    'Senin': 'Monday',
                    'Selasa': 'Tuesday',
                    'Rabu': 'Wednesday',
                    'Kamis': 'Thursday',
                    'Jumat': 'Friday',
                    'Sabtu': 'Saturday',
                    'Minggu': 'Sunday',
                    'Januari': 'January',
                    'Februari': 'February',
                    'Maret': 'March',
                    'April': 'April',
                    'Mei': 'May',
                    'Juni': 'June',
                    'Juli': 'July',
                    'Agustus': 'August',
                    'September': 'September',
                    'Oktober': 'October',
                    'November': 'November',
                    'Desember': 'December'
                }

                # Remove the leading hyphen from the date string
                date_text = date_text.replace('- ', '')

                # Replace the Indonesian day and month names with English names
                for ind_name, eng_name in translation_dict.items():
                    date_text = date_text.replace(ind_name, eng_name)

                # Parse the date string using the specified format
                date_obj = datetime.strptime(date_text, '%A, %d %B %Y | %H:%M WIB')

                # Format the date object as 'yyyy-mm-dd'
                formatted_date = date_obj.strftime('%Y-%m-%d')

                # Content Berita
                body_elem = soup.find('article', {"class": "read__content clearfix"})
                content_text = ""

                if body_elem:
                    content_elem = body_elem.find_all('p')
                    for p in content_elem:
                        content_text += p.text.strip() + "\n"

                    if content_text.strip():
                        content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                        content_text = ' '.join(content_text.split())
                    else:
                        content_text = "Content not found"
                else:
                    content_text = "Content not found"

                return {
                    'title': title_text,
                    'date': formatted_date,
                    'content': content_text,
                    'link': url
                }

            elif response.status_code == 429:
                print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                time.sleep(5)
            else:
                print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")

        except requests.exceptions.RequestException as e:
            print(f"Error fetching URL '{url}': {e}")
        except Exception as e:
            print(f"Error processing URL '{url}': {e}")

        retries += 1
        if retries < max_retries:
            print(f"Retrying {url} (Attempt {retries}/{max_retries})")
            time.sleep(5)  # You can adjust the delay as needed

    return None


In [8]:
url = 'https://www.krjogja.com/bantul/1242453743/mpls-sman-1-banguntapan-kenalkan-isu-kependudukan-kepada-siswa'
scrape_url(url)

{'title': 'MPLS SMAN 1 Banguntapan, Kenalkan Isu Kependudukan kepada Siswa',
 'date': '2023-07-19',
 'content': 'Krjogja.com - BANTUL - SMAN 1 Banguntapan Kabupaten Bantul yang pernah meraih juara 1 Sekolah Siaga Kependudukan (SKK) Paripurna Tingkat Nasional (2021) memasukkan materi kependudukan dalam kegiatan Masa Pengenalan Lingkungan Sekolah (MPLS). Kegiatan tersebut masuk dalam format \'Bincang Asyik bersama Bunda GenRe Kab Bantul\'.Dalam keterangan pernya, Rabu (19/07/2023), Humas SMA 1 Banguntapan menyebutkan, turut hadir dalam kegiatan ini Panewu Banguntapan, I Nyoman Gunarsa SPsi MPsi, Forum Genre (Generasi Berencana) Bantul, M Hengki Setiawan SE MM dan Kepala SMAN 1 Banguntapan Ziadatul Fauziah Aryati, dewan guru dan Kader PIK-R SSK Parahita serta OSIS Tatagawa SMAN 1 BanguntapanBunda Genre, Hj Emi Masruroh SPd yang juga istri Bupati Bantul, H Abdul Halim Muslih menekankan pentingnya pengenalan isu kependudukan kepada siswa untuk mewujudkan Bantul sebagai Kabupaten Layak Anak 